In [1]:
import json
import pandas as pd
import datetime
import os
import pyodbc

pd.set_option('display.max_columns',500)

os.chdir(r'N:\Python')

dpath = r"N:\Python\Inter Pipeline Semi-Quantitative Risk Algorithm.data.config"

server = 'sql2017'
config_db = 'ipl_irasv6_config'

now_str = datetime.datetime.now().strftime('%Y-%m-%d')
name = dpath.split("\\")[-1].split(".")[0][:20]

def get_config_details(db,server):

    driver = '{SQL Server Native Client 11.0}'

    conn = pyodbc.connect("Driver="+driver+";Server="+server+";Database="+db+";Trusted_Connection=yes;")

    q0 = """--**************************************************************************
        --PLAINS_IRASV6_CONFIG @ SQL2017
        --This query gathers additional information for the configured fields in the specified database.
        --**************************************************************************
        SELECT --VariableGroupId,
               VariableGroupName,
               --VariableCategoryParentId,
               VariableCategoryParentName,
               --VariableCategoryId,
               VariableCategoryName,
	           TableName,
	           ColumnName,
               --vari.VariableId,
               VariableName,
               Replace( VariableName, '_', ' ' ) AS VariableNameToUpdate,
               VariableDisplayText,
               /*HasVariableMetaData,
               VariableDataType,
               VariableDecimalCount,
               VariableUnitCode,
               HasDatabaseMetaData,*/
	           DataSourceDataType,
               DataTableType/*,
               DataSourceDecimalCount,
               DataSourceUnitCode*/
        FROM   dbo.VariableFieldXRef vx
               INNER JOIN ( SELECT dst.Id    AS DataSourceTableId,
                                   dst.NAME  AS TableName,
                                   dsf.Id    AS DataSourceFieldId,
                                   FieldName AS ColumnName,
                                   dt.NAME   AS DataSourceDataType,
                                   --dm.Id     AS HasDatabaseMetaData,
                                   CASE
                                     WHEN dst.TableType = 2 THEN 'Point'
                                     WHEN dst.TableType = 1 THEN 'Linear'
                                     WHEN dst.TableType = 3 THEN 'Virtual'
                                     ELSE 'Unknown'
                                   END       AS DataTableType--,
                                   --Code      AS DataSourceUnitCode,
                                   --PRECISION AS DataSourceDecimalCount
                            FROM   dbo.DataSourceTable dst
                                   LEFT JOIN dbo.DataSourceField dsf
                                          ON dst.Id = dsf.TableId
                                   --LEFT JOIN dbo.DataSourceFieldMetaData dm
                                   --       ON dsf.Id = dm.FieldId
                                   --LEFT JOIN dbo.UnitOfMeasure uom
                                   --       ON dm.UnitId = uom.Id
                                   LEFT JOIN dbo.DataType dt
                                          ON dsf.DataTypeId = dt.Id ) db
                       ON vx.DataSourceFieldId = db.DataSourceFieldId
               INNER JOIN ( SELECT vg.Id                AS VariableGroupId,
                                   vg.NAME              AS VariableGroupName,
                                   vcp.Id               AS VariableCategoryParentId,
                                   vcp.NAME             AS VariableCategoryParentName,
                                   vc.Id                AS VariableCategoryId,
                                   vc.NAME              AS VariableCategoryName,
                                   v.Id                 AS VariableId,
                                   v.NAME               AS VariableName,
                                   v.DisplayText        AS VariableDisplayText,
                                   dt.NAME              AS VariableDataType--,
                                   --vm.Id                AS HasVariableMetaData,
                                   --vm.DecimalPlaceCount AS VariableDecimalCount,
                                   --uom.Code             AS VariableUnitCode
                            FROM   dbo.Variable v
                                   LEFT JOIN dbo.DataType dt
                                          ON v.DataTypeId = dt.Id
                                   --LEFT JOIN dbo.VariableMetadata vm
                                   --       ON v.Id = vm.VariableId
                                   --LEFT JOIN dbo.UnitOfMeasure uom
                                   --       ON vm.UnitCodeId = uom.Id
                                   LEFT JOIN dbo.VariableGroup vg
                                          ON v.GroupId = vg.Id
                                   LEFT JOIN dbo.VariableCategory vc
                                          ON v.CategoryId = vc.Id
                                   LEFT JOIN dbo.VariableCategory vcp
                                          ON vc.ParentCategoryId = vcp.Id ) vari
                       ON vx.VariableId = vari.VariableId
        --WHERE  VariableGroupName IN ( 'VirtualTable' )
        ORDER  BY DatasourceTableId--VariableGroupName,VariableCategoryParentName,VariableCategoryName,TableName,ColumnName
        """
    df = pd.read_sql_query(q0,conn)

    conn.close()

    return df

def conv_list_to_str(l):
    o = ''
    for x in l:
        o = o+str(x)+'\n'
    return o.encode()

def parse_data_type_in(number):
    if number == 1:
        return 'Numeric'
    elif number == 2:
        return 'Datetime'
    elif number == 3:
        return 'String'
    else:
        pass

def parse_conf_var_in(string):
    if not(string):
        return 'Data Configuration Variable.'
    else:
        pass

In [2]:
def recursive_iter(obj, keys=()):
    if isinstance(obj,dict):
        for k,v in obj.items():
            yield from recursive_iter(v, keys+ (k,))
    elif any(isinstance(obj,t) for t in (list,tuple)):
        for idx, item in enumerate(obj):
            yield from recursive_iter(item, keys + (idx,))
    else:
        yield keys, obj

def recursive_iter2(obj):
    if isinstance(obj,dict):
        for k,v in obj.items():
            yield from recursive_iter(v, keys+ (k,))
    elif any(isinstance(obj,t) for t in (list,tuple)):
        for idx, item in enumerate(obj):
            yield from recursive_iter(item, keys + (idx,))
    else:
        yield keys, obj

print('loading data...')
with open(dpath,'r') as read_file:
    data = json.load(read_file)
print('json loaded.')

# for item in recursive_iter(data['SegmentationGroups']):
#     print(item)

loading data...
json loaded.


In [6]:
for x in data['SegmentationGroups']:
    print(x['Name'])

ValveLength
Final_Segmentation_Group
ProductComponent
InlineInspectionMFL
InlineInspectionSCC
InlineInspectionLFERW
Incident_UnauthorizedActivity
ECPOELeak
ECPOE_Rupture
ICPOE_Leak
ICPOE_Rupture
SCCSimulation_Leak
SCCSimulation_Rupture
LFERWSimulation_Leak
LFERWSimulation_Rupture
WatercrossingBankWidth
Natural Segmentation


In [16]:
for x in data['SegmentationGroups']:
    print(x['Name'])
    for y in x['Variables']:
        print('\t',y['Name'])
        for z in y['DataVariables']:
            print('\t\t',z['Variable'])
            for w in recursive_iter(z["Filters"]):
                print('\t\t\t',w)
                

ValveLength
	 Valve_Length_m
		 SegmentLength
	 Minimum_Valve_Length_Elevation_m
		 Elevation
	 Median_Valve_Length_Elevation_m
		 Elevation
	 Maximum_Valve_Length_Elevation_m
		 Elevation
	 Downstream_Valve_Operation_Type
		 Valve_Operator
Final_Segmentation_Group
	 Dynamic_Segment_Elevation_m
		 Elevation
ProductComponent
	 H2S_molfraction
		 ProductFlowRate
			 ((0, 'Node', 'LeftOperand', 'ListDomainProperty'), 'Code')
			 ((0, 'Node', 'LeftOperand', 'Variable'), 'Product_Component')
			 ((0, 'Node', 'Operator'), 'Equal')
			 ((0, 'Node', 'RightOperand', 'Value'), 'H2S')
	 Gas_Flow_Rate_m3perday
		 ProductFlowRate
			 ((0, 'Node', 'LeftOperand', 'ListDomainProperty'), 'Code')
			 ((0, 'Node', 'LeftOperand', 'Variable'), 'Product_Component')
			 ((0, 'Node', 'Operator'), 'Equal')
			 ((0, 'Node', 'RightOperand', 'Value'), 'Dry Natural Gas')
	 Oil_Flow_Rate_m3perday
		 ProductFlowRate
			 ((0, 'Node', 'LeftOperand', 'ListDomainProperty'), 'Code')
			 ((0, 'Node', 'LeftOperand', 'Varia